<a href="https://colab.research.google.com/github/tomonari-masada/course2025-sml/blob/main/11_document_clustering_%E6%8E%88%E6%A5%AD%E4%B8%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング
* クラスタリングの代表的な手法であるk平均法を使ってみる。
* ついでに、言語モデルを使ったテキストマイニングを体験してみる。

## 例題: 文書クラスタリング

* Transformerベースの日本語対応言語モデルを使って、テキストのベクトル表現を得る。
  * Transformerというニューラルネットワークについては、いずれ学びます。
  * 有名な解説記事 https://jalammar.github.io/illustrated-transformer/
* テキストをベクトルとして表現することを「embedする」と言う。
  * embedすることで得られるベクトルのことを「embedding」と言う。
* そして、テキストのembeddingをk平均法でクラスタリングする。

* ランタイムのタイプをGPUにしておく。

## インストール

### spaCyの日本語モデル

* 日本語テキストを形態素解析するために使う。
  * たぶん、セッションの再起動」は不要。

In [ ]:
!python -m spacy download ja_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Hugging Faceのdatasetsライブラリ

* ライブドアニュースコーパスを取得するために使う。

In [ ]:
!pip install --upgrade datasets huggingface_hub

### SentenceTransformersライブラリ
* 言語モデルを使ってテキストを埋め込む際に便利なライブラリ。
  * https://sbert.net/index.html

In [ ]:
!pip install -U sentence-transformers

## インポート

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import spacy

from datasets import load_dataset
from transformers import set_seed
from sentence_transformers import SentenceTransformer

# 再現性の確保
set_seed(1234)

## データセット
* livedoorニュースコーパスを使う。

In [ ]:
dataset = load_dataset(
  "shunk031/livedoor-news-corpus",
  train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
  random_state=42,
  shuffle=True,
  trust_remote_code=True,
)

num_categories = len(set(dataset["train"]["category"]))

category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

print(f"num_categories: {num_categories}")
print(f"category_names: {category_names}")

num_categories: 9
category_names: ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']


In [ ]:
dataset["train"][0]

{'url': 'http://news.livedoor.com/article/detail/5834377/',
 'date': '2011-09-04T08:30:00+0900',
 'title': '【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露',
 'content': '10月、東京で行われる世界体操では、兄・田中和仁＆弟・田中佑典とともに3兄弟で出場を決めた田中理恵。2日、フジテレビ「すぽると！」では「田中3兄弟SP対談」と題し、兄弟3人によるトークの模様が放送された。  「お兄ちゃんは、形にはまって美しい体操をしている感じがする。教科書に載るような。佑典は綺麗で、かつオシャレやなって一言言いたくなる演技」、その他にも、「ずっと（兄弟を）見ていたから、他の女子の“こういう選手になりたい”というのがない。弟の鉄棒みたいなオシャレ演技したいし、お兄ちゃんみたいな綺麗な線出したいしという気持ちが強い」と語った理恵。  対して、理恵が“美人アスリート”と呼ばれることについて、和仁は「妹が出てるってことで、兄としては嬉しいけど、色んな人に言われても“ふーん”っていって終わり」と素っ気なく、佑典は「美人アスリート。まあ、不細工アスリートよりは、美人アスリートなんじゃないですかね」と語る。  また、理恵の学生時代の様子について、「和歌山の（実家の）時は部屋汚かったです」と明かす和仁。これには理恵も「ここで言う？」と呆れたが、和仁は、お構いなしに「高校の時、髪染めたな。“染めてない”って言ってたけど、染めてたな。で、（父・章二さんから）リモコン飛んできたんだよな。おとんが教師なのに、そこの生徒でよう染めたなっていうのは、皆思ってたよ」と暴露を続けた。  ・田中理恵 写真ギャラリー',
 'category': 6}

In [ ]:
dataset["train"]["title"][:10]

['【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露',
 '「美人で何が悪い！？」 負け美女・小島慶子×犬山紙子対談 3/3',
 'もうひとつのアカデミー賞!?\u3000スカパーアダルト放送大賞が決定！',
 '渡邉美樹氏「猫ひろしさんのカンボジア国籍取得 納得いきません」',
 'やくみつるさんの「DeNAが許せない」発言に批判殺到',
 'ポスターのテーマは、剣心の“封印された狂気”',
 '悩ましき女友だちとの格差問題',
 'ラジオ体操は「究極のエクササイズ」ってホント？',
 'オトナ女子たちの圧倒的支持をうけ、ドラマ10『はつ恋』の一挙再放送が決定！',
 '映画『サルベージ・マイス』主題歌に、ももいろクローバーＺの新曲が決定']

In [ ]:
dataset["train"]["content"][0]

'10月、東京で行われる世界体操では、兄・田中和仁＆弟・田中佑典とともに3兄弟で出場を決めた田中理恵。2日、フジテレビ「すぽると！」では「田中3兄弟SP対談」と題し、兄弟3人によるトークの模様が放送された。  「お兄ちゃんは、形にはまって美しい体操をしている感じがする。教科書に載るような。佑典は綺麗で、かつオシャレやなって一言言いたくなる演技」、その他にも、「ずっと（兄弟を）見ていたから、他の女子の“こういう選手になりたい”というのがない。弟の鉄棒みたいなオシャレ演技したいし、お兄ちゃんみたいな綺麗な線出したいしという気持ちが強い」と語った理恵。  対して、理恵が“美人アスリート”と呼ばれることについて、和仁は「妹が出てるってことで、兄としては嬉しいけど、色んな人に言われても“ふーん”っていって終わり」と素っ気なく、佑典は「美人アスリート。まあ、不細工アスリートよりは、美人アスリートなんじゃないですかね」と語る。  また、理恵の学生時代の様子について、「和歌山の（実家の）時は部屋汚かったです」と明かす和仁。これには理恵も「ここで言う？」と呆れたが、和仁は、お構いなしに「高校の時、髪染めたな。“染めてない”って言ってたけど、染めてたな。で、（父・章二さんから）リモコン飛んできたんだよな。おとんが教師なのに、そこの生徒でよう染めたなっていうのは、皆思ってたよ」と暴露を続けた。  ・田中理恵 写真ギャラリー'

## 多言語E5による埋め込み

* Multilingual E5を使う。
  * テキストのembeddingにおいて優れている言語モデル。
  * 論文 https://arxiv.org/abs/2402.05672
  * Hugging Face https://huggingface.co/intfloat/multilingual-e5-large-instruct

* 参考: テキスト埋め込みのleaderboard
  * https://huggingface.co/spaces/mteb/leaderboard

* SentenceTransformerを使ったテキストの埋め込みについては、下のWebページを参照。
  * https://sbert.net/examples/sentence_transformer/applications/computing-embeddings/README.html

In [ ]:
model_id = "intfloat/multilingual-e5-large-instruct"
model = SentenceTransformer(model_id)

* 試しに、一つだけ、テキストを埋め込んでみる。

In [ ]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
model.encode(dataset["train"][0]["title"])

array([ 0.01802401,  0.03642237, -0.00951136, ..., -0.03236501,
       -0.03817462,  0.01583127], dtype=float32)

* ライブドアニュースコーパスの全タイトルを埋め込む。

In [ ]:
embeddings = model.encode(dataset["train"]["title"], show_progress_bar=True)

Batches:   0%|          | 0/185 [00:00<?, ?it/s]

* 埋め込みは普通にNumPyの配列として得られている。

In [ ]:
type(embeddings)

numpy.ndarray

In [ ]:
embeddings.shape

(5894, 1024)

* 全記事内容を埋め込むには以下のようにする。  
  * RTX3080搭載PCを使うと1分で終わる。

In [ ]:
#content_embeddings = model.encode(dataset["train"]["content"], show_progress_bar=True)

* ただし、どのテキストも先頭から512トークンで切られていることに注意。
  * 長いテキストは、途中までの内容しかembeddingに反映されない。
  * それでも、分類やクラスタリングがうまくいくことも多い。

In [ ]:
model.max_seq_length

512

* トークン数の調べ方
  * トークナイザにテキストを分割させる。
  * 分割によって得られたトークンの個数を数える。

In [ ]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [ ]:
model.tokenize([dataset["train"][0]["title"]])

{'input_ids': tensor([[     0,   5946,  43488,      7,  20413,   2728,   4742,  40019,   1925,
            6676,    514,   6986, 126235,     37,  67156,    281,   7252, 162829,
           11264,  36948, 124408,   7116,    251, 174856,      2]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1]])}

In [ ]:
(model.tokenize([dataset["train"][0]["title"]])['input_ids']).shape[1]

25

* 埋め込みを保存。

In [ ]:
with open('embeddings.npy', 'wb') as f:
  np.save(f, embeddings)

In [ ]:
#with open('content_embeddings.npy', 'wb') as f:
#  np.save(f, content_embeddings)

* 読み込みは以下のようにする。

In [ ]:
with open('embeddings.npy', 'rb') as f:
  embeddings = np.load(f)

In [ ]:
#with open('content_embeddings.npy', 'rb') as f:
#  content_embeddings = np.load(f)

## クラスタのラベリングに使う単語の抽出

* 全テキストを形態素解析する。
  * 形態素解析＝単語への分割

In [ ]:
nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(dataset["train"]["title"]):
  corpus.append(" ".join([token.lemma_ for token in nlp(text)]))

  0%|          | 0/5894 [00:00<?, ?it/s]

In [ ]:
dataset["train"][0]["title"]

'【Sports Watch】体操・田中理恵、兄が学生時代のエピソードを暴露'

In [ ]:
corpus[0]

'【 SPORTS watch 】 体操 ・ 田中 理恵 、 兄 が 学生 時代 の エピソード を 暴露'

* scikit-learnでTF-IDFを計算する。
* `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [ ]:
vectorizer = TfidfVectorizer(min_df=20)

In [ ]:
X_train = vectorizer.fit_transform(corpus)

In [ ]:
X_train = X_train.toarray()

In [ ]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X_train.shape

(5894, 447)

In [ ]:
vocab = np.array(vectorizer.get_feature_names_out())

In [ ]:
vocab.size

447

In [ ]:
print(list(vocab))

['01', '02', '04', '05', '06', '07', '10', '100', '1000万', '11', '12', '13', '15', '1日', '20', '2011', '2012', '23', '24', '30', '40', '48', 'akb', 'android', 'aquos', 'arrows', 'au', 'by', 'cafe', 'cm', 'cpu', 'dvd', 'facebook', 'fi', 'galaxy', 'google', 'hd', 'ics', 'ipad', 'iphone', 'is', 'isw', 'it', 'kddi', 'lte', 'mac', 'medias', 'nhk', 'note', 'ntt', 'optimus', 'os', 'pc', 'phone', 'presented', 'salon', 'sc', 'sh', 'sports', 'twitter', 'vol', 'vs', 'watch', 'wi', 'wimax', 'windows', 'xi', 'xperia', 'あなた', 'あの', 'ある', 'いい', 'いう', 'いく', 'いる', 'おく', 'お気に入り', 'から', 'かわいい', 'くる', 'くれる', 'こと', 'この', 'これ', 'さん', 'しまう', 'すぎる', 'する', 'せる', 'そう', 'その', 'たい', 'たち', 'ため', 'だけ', 'ちゃう', 'ちゃん', 'って', 'っと', 'ついに', 'つく', 'てる', 'できる', 'です', 'どう', 'どこ', 'ない', 'なし', 'なぜ', 'なでしこ', 'など', 'なる', 'なん', 'にて', 'にゅう', 'べし', 'ます', 'まで', 'まとめ', 'みる', 'みんな', 'もう', 'もの', 'やすい', 'やる', 'ゆるい', 'よう', 'より', 'よる', 'られる', 'れる', 'わかる', 'アイテム', 'アップ', 'アップル', 'アナ', 'アプリ', 'イケショップ', 'イベント', 'インタビュー', 'インチ', 'オススメ', 'オトナ

## ラベリング用単語の埋め込み

* 各単語について、その単語を含むテキストの埋め込みベクトルの加重平均を求める。
* 加重平均の重みは、各テキストにおけるその単語のTF-IDFの値を使って定める。

In [ ]:
X_train.sum(0)

array([ 14.23737419,   8.29165953,   8.89160565,  10.98194494,
        18.93665008,   9.46061414,  40.07367426,  14.28822988,
        14.41911992,  22.80643974,  18.05936778,  11.4279328 ,
        11.59444793,  11.84556731,  20.7174215 ,  23.58182266,
        41.04647437,  12.39391169,  10.63096185,  17.4181583 ,
        10.99017477,  27.09842548,  35.27405367,  78.35690806,
        15.26955653,  10.83606064,  24.80076668,  23.88017003,
        20.80809394,  14.17248708,  10.96561034,  20.0860085 ,
        18.03084136,  14.36513671,  24.92943815,  22.90955665,
        10.09431811,  12.73046169,  24.04995493,  76.33793521,
         7.58491082,   8.07353078,  15.39973941,  16.2993322 ,
        12.22679908,  10.5302457 ,   9.04033087,  12.29946058,
         8.40654504,  43.04142467,   8.86133668,   9.48350301,
        23.8057352 ,  21.52179432,  19.95833812,  29.16454774,
        13.14260003,  18.92114609, 154.76217334,  14.14442497,
        72.94053245,  12.27233884, 154.76217334,  14.92

In [ ]:
text_weights = X_train / X_train.sum(0)

In [ ]:
vocab_embeddings = np.dot(text_weights.T, embeddings)

## 文書クラスタリング



In [ ]:
embeddings.shape

(5894, 1024)

* k-meansのしくみ

* 初期化

In [ ]:
n_clusters = 20
assignments = np.random.randint(0, n_clusters, X_train.shape[0])

In [ ]:
assignments

array([15, 19,  6, ..., 14,  6,  2])

* クラスタの重心の計算

In [ ]:
mean_vectors = []
for k in range(n_clusters):
  mean_vectors.append(embeddings[assignments == k].mean(0))

In [ ]:
mean_vectors = np.array(mean_vectors)

In [ ]:
mean_vectors

array([[ 0.01647943,  0.02976647, -0.01883664, ..., -0.01935209,
        -0.02470126,  0.00820278],
       [ 0.01736941,  0.02942751, -0.01961963, ..., -0.01800565,
        -0.02569819,  0.01002786],
       [ 0.01818474,  0.02903217, -0.01919492, ..., -0.01931877,
        -0.02524555,  0.00853535],
       ...,
       [ 0.01856   ,  0.02957751, -0.01961546, ..., -0.01758128,
        -0.02471472,  0.00871525],
       [ 0.01793541,  0.02967603, -0.02068584, ..., -0.01822105,
        -0.02434707,  0.00888514],
       [ 0.01804006,  0.029566  , -0.01940084, ..., -0.0179777 ,
        -0.02628745,  0.00799878]], dtype=float32)

* 各ベクトルに最も近い重心ベクトルを見つけて、クラスタを割り当て直す

In [ ]:
np.sqrt(((embeddings[0] - mean_vectors[0]) ** 2).sum())

np.float32(0.40364903)

In [ ]:
distances = []
for k in range(n_clusters):
  distances.append(np.linalg.norm(embeddings[0] - mean_vectors[k]).item())
distances

[0.4036490321159363,
 0.409040629863739,
 0.4111645221710205,
 0.4118451476097107,
 0.4051388204097748,
 0.40543273091316223,
 0.41023483872413635,
 0.4078647494316101,
 0.4158936142921448,
 0.4042888581752777,
 0.40534093976020813,
 0.40762612223625183,
 0.4092589318752289,
 0.4115009903907776,
 0.4054745137691498,
 0.40657368302345276,
 0.4127759337425232,
 0.4144781231880188,
 0.41011178493499756,
 0.40817224979400635]

### k-平均法によるクラスタリング

In [ ]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
#kmeans.fit(content_embeddings) # 本文の場合はこちら。
centers = kmeans.cluster_centers_

In [ ]:
centers

array([[ 0.01430322,  0.03136253, -0.02514517, ..., -0.02268711,
        -0.0266082 ,  0.01325098],
       [ 0.0189735 ,  0.02687819, -0.02668014, ..., -0.01993991,
        -0.01975193,  0.01015017],
       [ 0.01751033,  0.02260082, -0.02669384, ..., -0.01917516,
        -0.02717998,  0.01307425],
       ...,
       [ 0.02170468,  0.02648552, -0.02336103, ..., -0.01304382,
        -0.0181316 ,  0.00705924],
       [ 0.01176104,  0.02763935, -0.02222423, ..., -0.02498448,
        -0.0299366 ,  0.01751282],
       [ 0.02082054,  0.0304917 , -0.02854665, ..., -0.02131261,
        -0.02837157,  0.01084703]], dtype=float32)

* クラスタの重心を保存。

In [ ]:
with open(f'centers_{n_clusters}.npy', 'wb') as f:
  np.save(f, centers)

In [ ]:
with open(f'centers_{n_clusters}.npy', 'rb') as f:
  centers = np.load(f)

### クラスタのサイズを調べる

* クラスタのインデックスをキーとし、そのサイズを値とする辞書を作る。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique.tolist(), counts.tolist()))

* 辞書のエントリを、キーではなく値でソートする。

In [ ]:
print([sorted(size_dict.items(), key=lambda item: item[1], reverse=True)])

[[(4, 498), (13, 450), (8, 388), (3, 360), (14, 346), (5, 343), (19, 343), (9, 341), (10, 320), (11, 290), (7, 287), (0, 280), (1, 280), (15, 273), (17, 236), (2, 211), (16, 197), (6, 180), (18, 154), (12, 117)]]


## クラスタのラベリング
* 各クラスタの重心に近い単語でラベリングする。

* テキストの埋め込みは、長さ1のベクトルになっている。

In [ ]:
np.linalg.norm(embeddings, axis=-1)

array([1.        , 1.        , 0.99999994, ..., 1.        , 1.        ,
       1.        ], dtype=float32)

* テキストとラベリング用の単語との類似度はコサイン類似度で測る。

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vocab_embeddings, centers)

In [ ]:
vocab_embeddings.shape

(447, 1024)

In [ ]:
centers.shape

(20, 1024)

In [ ]:
similarities.shape

(447, 20)

* 重心に近い順に30個の単語を表示する。

In [ ]:
for i in range(similarities.shape[-1]):
  indices = np.argsort(- similarities[:,i])
  print(vocab[indices[:30]])

['どう' 'って' 'あなた' 'いる' 'なぜ' '本当' '事情' '男性' '好き' 'なる' 'ある' '女性' '思う' 'しまう'
 'たい' 'どこ' 'もの' 'その' 'こと' '聞く' 'たち' 'れる' '自分' 'べし' '必要' 'いう' 'ない' '女子'
 '彼女' '合う']
['デジ' '使える' '使う' '機能' '携帯' 'スマホ' 'ipad' '利用' 'iphone' '可能' 'できる' '端末'
 'アップル' '情報' '高い' '専用' '話題' 'みる' 'サービス' '発表' 'ユーザー' '新しい' 'から' 'ニュース' '開発'
 '紹介' 'it' 'わかる' '登場' 'おく']
['スマートフォン' 'レポート' 'スマホ' '対応' '06' '05' 'galaxy' 'コンパクト' '試す' '搭載' '01'
 'ドコモ' 'モバイル' '入り' '画面' '向け' '全部' 'レビュー' 'optimus' '防水' '04' '機能' '動画'
 'phone' 'sh' 'モデル' '通信' '07' 'sc' '発売']
['恋愛' 'しまう' '女子' 'たち' 'たい' '事情' '独女' '男性' '女性' '合う' 'いる' '好き' '彼女' 'なる'
 '思う' 'モテる' 'こと' 'れる' '自分' '悩み' 'られる' '聞く' 'せる' '本当' '結婚' '見る' 'って' 'vol'
 '男子' '幸せ']
['映画' '公開' '映像' '解禁' '作品' '特別' 'まとめ' '最高' '予告' 'スター' '決定' '編集' '最強' '誕生'
 '読み' '世界' '上陸' '週末' 'dvd' '挑戦' '込む' 'ドラマ' '国際' 'れる' 'する' '注目' '少女' 'から'
 'られる' '批評']
['さん' 'ちゃん' '人気' 'れる' 'する' 'くる' 'すぎる' 'いい' 'なる' 'あの' 'から' 'いく' 'いる' '見る'
 'られる' '込む' 'たい' 'もう' 'てる' 'せる' 'ちゃう' '出演' 'ある' '出す' 'ない' 'いう' '出る' 'ます'
 '注目' 'この']
['仕事' '会社' 'v

# プランナー課題１１
* それぞれのクラスタについて、重心に近い元々のテキスト（つまり記事タイトル）を5件ずつ表示させてみよう。
* それらのテキストの内容に、上で得たラベルが合っているかどうか、確かめよう。